In [2]:
import pandas as pd
import numpy as np

In [3]:
df_product = pd.read_csv('Assets/data/Hackathon_DimProduct_SAN_vShared.csv',header=0)
df_product

,ProductKey,BrandKey,SupplierKey,ProductCategory_Lvl1,ProductCategory_Lvl2
0,key_52138,3521,1039040101,Category A,category aa
1,19864,3521,1039040125,Category A,Category AC
2,95144,1713,1049688101,Category A,Category AA
3,23070,3521,1050235101,Category A,Category AC
4,93165,3521,1039040125,Category A,Category AC
...,...,...,...,...,...
293,12353,3521,1036290101,Category A,Category AB
294,55442,132,1036290101,Category A,Category AC
295,78206,3521,1039040101,Category A,Category AC
296,55438,132,1036290101,Category A,Category AB


In [44]:
df_product.dtypes

ProductKey              object
BrandKey                 int64
SupplierKey              int64
ProductCategory_Lvl1    object
ProductCategory_Lvl2    object
dtype: object

In [52]:
import string
import re

def remove_punctuations(ts):
    '''Takes a word and removes all punctuations.
     :parameter
     ts: str
        the word to be cleaned
    :returns
    -----
    str
        the cleaned version
      '''
    ts=str(ts)
    for punctuation in string.punctuation:
        ts = ts.replace(punctuation, '')
    return ts

def remove_non_numbers(ts):
    '''Takes a word and removes all non-digit characters.
     :parameter
     ts: str
        the word to be cleaned
    :returns
    -----
    str
        the cleaned version
      '''
    return int(re.sub('[^0-9.]', '',str(ts)))

def name_corrector(ts):
    '''Takes a word and removes all whitespaces, transforms it to lowercase,
     and connects the words using '_'. Also checks for misspelling.
     :parameter
     ts: str
        the word to be corrected
    :returns
    -----
    str
        the corrected version
      '''
    return re.sub('categrory','category','_'.join(re.sub(' +',' ',ts).lower().split(' ')))

In [45]:
df_product.columns

Index(['ProductKey', 'BrandKey', 'SupplierKey', 'ProductCategory_Lvl1',
       'ProductCategory_Lvl2'],
      dtype='object')

In [51]:
df_product_clean = df_product.copy()
for col in ['ProductKey', 'BrandKey', 'SupplierKey']:
    df_product_clean[col] = df_product[col].apply(remove_punctuations)
    df_product_clean[col] = df_product[col].apply(remove_non_numbers)

for col in ['ProductCategory_Lvl1','ProductCategory_Lvl2']:
    df_product_clean[col]= df_product[col].apply(remove_punctuations)
    df_product_clean[col] = df_product[col].apply(name_corrector)


In [68]:
df_promotion = pd.read_csv('Assets/data/Hackathon_DimPromotion_SAN_vShared.csv',header=0)
df_promotion

,PromotionKey,PromoMechanic,PromotionStartDate,PromotionEndDate
0,312237,Percentage off,10/1/2021,10/6/2021
1,265157,Buy $X for Y%,2/25/2020,2/25/2020
2,270553,Buy $X for $Y,1/15/2020,1/27/2020
3,308082,X units for $Y (X = 1),3/10/2021,3/29/2021
4,317323,PWP,5/13/2022,7/6/2022
...,...,...,...,...
3426,297974,X units for $Y (X = 1),1/1/2022,6/30/2022
3427,339287,Buy $X for $Y,12/12/2021,12/12/2021
3428,263837,X units for $Y (X = 1),2/27/2020,3/30/2020
3429,375529,X units for $Y (X > 1),11/16/2022,1/2/2023


In [61]:
df_promotion.dtypes

PromotionKey           int64
PromoMechanic         object
PromotionStartDate    object
PromotionEndDate      object
dtype: object

In [69]:
df_promotion.PromotionStartDate = df_promotion.PromotionStartDate.astype('datetime64[ns]')
df_promotion.PromotionEndDate = df_promotion.PromotionEndDate.astype('datetime64[ns]')

In [70]:
df_promotion.dtypes

PromotionKey                   int64
PromoMechanic                 object
PromotionStartDate    datetime64[ns]
PromotionEndDate      datetime64[ns]
dtype: object

In [72]:
df_store = pd.read_csv('Assets/data/Hackathon_DimStore_SAN_vShared.csv',header=0)
df_store

,StoreKey,DistributionChannel,StoreType,Region_Lvl1,Region_Lvl2
0,559,Physical,Store Type B,RegionA,Region 34
1,168,Physical,Store Type B,RegionA,Region 2
2,2061,Physical,Store Type E,RegionA,Region 10
3,590,Physical,Store Type D,RegionA,Region 2
4,301,Physical,Store Type B,RegionA,Region 123
...,...,...,...,...,...
734,463,Physical,Store Type B,RegionB,Region 141
735,4,Physical,Store Type C,RegionA,Region 4
736,280,Physical,Store Type D,RegionB,Region 39
737,688,Physical,Store Type G,RegionA,Region 112


In [73]:
df_store.dtypes

StoreKey                int64
DistributionChannel    object
StoreType              object
Region_Lvl1            object
Region_Lvl2            object
dtype: object

In [74]:
df_store.DistributionChannel.unique()

array(['Physical', 'Online'], dtype=object)

In [75]:
df_store.StoreType.unique()

array(['Store Type B', 'Store Type E', 'Store Type D', 'Store Type A',
       'Store Type C', 'Store Type G', 'Other', 'Store Type F',
       'Store Type K', 'Store Type J', 'Store Type I', 'Store Type H'],
      dtype=object)

In [76]:
df_store.Region_Lvl1.unique()

array(['RegionA', 'RegionB', 'Online'], dtype=object)

In [77]:
df_store.Region_Lvl2.unique()

array(['Region 34', 'Region 2', 'Region 10', 'Region 123', 'Region 179',
       'Region 39', 'Region 158', 'Region 88', 'Region 7', 'Region 117',
       'Region 12', 'Region 114', 'Region 11', 'Region 55', 'Region 30',
       'Region 72', 'Region 217', 'Region 105', 'Region 6', 'Region 146',
       'Region 61', 'Region 44', 'Region 133', 'Region 51', 'Region 210',
       'Region 69', 'Region 106', 'Region 118', 'Region 208', 'Region 60',
       'Region 3', 'Region 35', 'Region 80', 'Region 74', 'Region 36',
       'Region 18', 'Region 161', 'Region 96', 'Region 174', 'Region 38',
       'Region 68', 'Region 153', 'Region 65', 'Region 8', 'Region 42',
       'Region 185', 'Region 20', 'Region 159', 'Region 99', 'Region 28',
       'Region 157', 'Unknown', 'Region 199', 'Region 98', 'Region 122',
       'Region 4', 'Region 173', 'Region 120', 'Region 186', 'Region 33',
       'Region 64', 'Region 125', 'Region 9', 'Region 67', 'Region 31',
       'Region 17', 'Region 21', 'Region 211', '

# checking the null values

In [78]:
print(df_store.isnull().sum())

StoreKey               0
DistributionChannel    0
StoreType              0
Region_Lvl1            0
Region_Lvl2            0
dtype: int64


In [79]:
print(df_promotion.isnull().sum())

PromotionKey          0
PromoMechanic         0
PromotionStartDate    0
PromotionEndDate      0
dtype: int64


In [80]:
print(df_product_clean.isnull().sum())

ProductKey              0
BrandKey                0
SupplierKey             0
ProductCategory_Lvl1    0
ProductCategory_Lvl2    0
dtype: int64


In [ ]:
import numpy as np
from nltk.corpus import stopwords
import spacy
nlp = spacy.load('it_core_news_sm')
import string
import re
import gensim
from gensim.utils import simple_preprocess
class preprocess_pip():
    def __init__(self,df ,column_name):
        self.df = df
        self.old_text = column_name
        self.text = column_name+'_clean'

    def apply_preprocessing(self):
#         self.df[self.text] =  self.df[self.old_text].astype('string')
        self.df[self.text] = self.df[self.old_text].apply(self.na_filter)
        self.df.dropna(subset=[self.text],inplace=True)
        self.df[self.text] = self.df[self.text].apply(self.remove_punctuations)
        self.df[self.text] = self.df[self.text].apply(self.to_lower)
        self.df[self.text] = self.df[self.text].apply(self.stop_word_elm)
        self.df[self.text] = self.df[self.text].apply(self.deEmojify)
        self.df[self.text] = self.df[self.text].apply(self.na_filter)
        self.df.dropna(axis=0, how='any',subset=[self.text])
#         self.df.drop('index',axis=1,inplace=True)
        self.df.reset_index(inplace=True)
        self.df.drop('index',axis=1,inplace=True)
        self.df[self.text].replace("[0-9,€$-;\'\"]", '', regex=True,inplace=True)
        self.df.dropna(subset=[self.text],inplace=True)
        self.df=self.df[self.df[self.text]!='']


    def na_filter(self, text):
        if len(text)>0:
            return text
        return

    def remove_punctuations(self, text):
        text=str(text)
        for punctuation in string.punctuation:
            text = text.replace(punctuation, '')
        return text

    def to_lower(self, text):
        return text.lower()

    def stop_word_elm(self, text):
        new_text=[]
        for i in text.split():
            if i not in italian_stopwords:
                new_text.append(i)
        return ' '.join(new_text)

    def deEmojify(self, text):
        regrex_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            "🤮"
            "🤬"
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',text)

    def sent_to_words(self, text):
        for sentence in text:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations